In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.937495231628418, Val Loss - 1.8065669536590576, Val Accuracy - 0.623
Epoch 20, Train Loss - 0.014776604250073433, Val Loss - 0.6284552812576294, Val Accuracy - 0.797
Epoch 40, Train Loss - 0.0071288710460066795, Val Loss - 0.6533370018005371, Val Accuracy - 0.792
Epoch 60, Train Loss - 0.017278265208005905, Val Loss - 0.6016064286231995, Val Accuracy - 0.804
Epoch 80, Train Loss - 0.016184350475668907, Val Loss - 0.6125009059906006, Val Accuracy - 0.814
Epoch 100, Train Loss - 0.012066339142620564, Val Loss - 0.6053387522697449, Val Accuracy - 0.812
Epoch 120, Train Loss - 0.013682592660188675, Val Loss - 0.6100842356681824, Val Accuracy - 0.811
Epoch 140, Train Loss - 0.01108565554022789, Val Loss - 0.6138620972633362, Val Accuracy - 0.815
Epoch 160, Train Loss - 0.009907764382660389, Val Loss - 0.6314827799797058, Val Accuracy - 0.803
Epoch 180, Train Loss - 0.011707643046975136, Val Loss - 0.6219707727432251, Val Accuracy - 0.816
Epoch 200, Train Loss - 0.010

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.803
Initial Loss: 0.6347781419754028


In [7]:
amts = defaultdict(int)
# Run 10 attacks for perturbation of 0.2
for i in range(10):
    model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)
    train = Trainable(model)
    train.fit(data, 200)
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)
    attacker.reset_parameters(seed=i)
    attacker.reset()
    attacker.attack(0.2)

    for edge in attacker._added_edges.keys():
        amts[edge] += 1


Epoch 0, Train Loss - 1.940118670463562, Val Loss - 1.794613003730774, Val Accuracy - 0.724
Epoch 20, Train Loss - 0.017813535407185555, Val Loss - 0.6152759790420532, Val Accuracy - 0.8
Epoch 40, Train Loss - 0.009128953330218792, Val Loss - 0.6565888524055481, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.013187751173973083, Val Loss - 0.6269016861915588, Val Accuracy - 0.794
Epoch 80, Train Loss - 0.014206903986632824, Val Loss - 0.5997770428657532, Val Accuracy - 0.815
Epoch 100, Train Loss - 0.011512398719787598, Val Loss - 0.6254796385765076, Val Accuracy - 0.804
Epoch 120, Train Loss - 0.010277456603944302, Val Loss - 0.6154866814613342, Val Accuracy - 0.809
Epoch 140, Train Loss - 0.008580419234931469, Val Loss - 0.6496993899345398, Val Accuracy - 0.796
Epoch 160, Train Loss - 0.009299584664404392, Val Loss - 0.6277406215667725, Val Accuracy - 0.809
Epoch 180, Train Loss - 0.01195374596863985, Val Loss - 0.6309424638748169, Val Accuracy - 0.815
Epoch 200, Train Loss - 0.0094510

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 0, Train Loss - 1.9534170627593994, Val Loss - 1.8369770050048828, Val Accuracy - 0.581
Epoch 20, Train Loss - 0.011925701051950455, Val Loss - 0.6612085103988647, Val Accuracy - 0.79
Epoch 40, Train Loss - 0.008358433842658997, Val Loss - 0.6876540184020996, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.011777459643781185, Val Loss - 0.6186111569404602, Val Accuracy - 0.802
Epoch 80, Train Loss - 0.013358676806092262, Val Loss - 0.6146355271339417, Val Accuracy - 0.807
Epoch 100, Train Loss - 0.009547621943056583, Val Loss - 0.6130049228668213, Val Accuracy - 0.812
Epoch 120, Train Loss - 0.01181965135037899, Val Loss - 0.6150628924369812, Val Accuracy - 0.809
Epoch 140, Train Loss - 0.0118984030559659, Val Loss - 0.6284156441688538, Val Accuracy - 0.806
Epoch 160, Train Loss - 0.010907652787864208, Val Loss - 0.6246521472930908, Val Accuracy - 0.804
Epoch 180, Train Loss - 0.009696303866803646, Val Loss - 0.6392356157302856, Val Accuracy - 0.804
Epoch 200, Train Loss - 0.008546

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 0, Train Loss - 1.9534170627593994, Val Loss - 1.8369770050048828, Val Accuracy - 0.581
Epoch 20, Train Loss - 0.01192570012062788, Val Loss - 0.6612085103988647, Val Accuracy - 0.79
Epoch 40, Train Loss - 0.008358433842658997, Val Loss - 0.6876540184020996, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.011777456849813461, Val Loss - 0.618611216545105, Val Accuracy - 0.802
Epoch 80, Train Loss - 0.013358674943447113, Val Loss - 0.6146355271339417, Val Accuracy - 0.807
Epoch 100, Train Loss - 0.009547621011734009, Val Loss - 0.6130045652389526, Val Accuracy - 0.812
Epoch 120, Train Loss - 0.011819645762443542, Val Loss - 0.6150641441345215, Val Accuracy - 0.809
Epoch 140, Train Loss - 0.011898423545062542, Val Loss - 0.6284164786338806, Val Accuracy - 0.806
Epoch 160, Train Loss - 0.010906434617936611, Val Loss - 0.6246225833892822, Val Accuracy - 0.804
Epoch 180, Train Loss - 0.009696985594928265, Val Loss - 0.6391854286193848, Val Accuracy - 0.804
Epoch 200, Train Loss - 0.00855

...

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 0, Train Loss - 1.9534170627593994, Val Loss - 1.8369770050048828, Val Accuracy - 0.581
Epoch 20, Train Loss - 0.01192570198327303, Val Loss - 0.66120845079422, Val Accuracy - 0.79
Epoch 40, Train Loss - 0.008358433842658997, Val Loss - 0.6876539587974548, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.01177745871245861, Val Loss - 0.6186111569404602, Val Accuracy - 0.802
Epoch 80, Train Loss - 0.013358676806092262, Val Loss - 0.6146355271339417, Val Accuracy - 0.807
Epoch 100, Train Loss - 0.009547621011734009, Val Loss - 0.6130048632621765, Val Accuracy - 0.812
Epoch 120, Train Loss - 0.011818959377706051, Val Loss - 0.615066647529602, Val Accuracy - 0.809
Epoch 140, Train Loss - 0.011898159049451351, Val Loss - 0.6283852458000183, Val Accuracy - 0.806
Epoch 160, Train Loss - 0.010911268182098866, Val Loss - 0.6246269345283508, Val Accuracy - 0.804
Epoch 180, Train Loss - 0.009692697785794735, Val Loss - 0.6392967700958252, Val Accuracy - 0.804
Epoch 200, Train Loss - 0.00855162

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

Epoch 0, Train Loss - 1.9534170627593994, Val Loss - 1.8369770050048828, Val Accuracy - 0.581
Epoch 20, Train Loss - 0.011925701051950455, Val Loss - 0.66120845079422, Val Accuracy - 0.79
Epoch 40, Train Loss - 0.008358435705304146, Val Loss - 0.6876540184020996, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.011777455918490887, Val Loss - 0.6186110973358154, Val Accuracy - 0.802
Epoch 80, Train Loss - 0.013358675874769688, Val Loss - 0.6146354079246521, Val Accuracy - 0.807
Epoch 100, Train Loss - 0.009547621011734009, Val Loss - 0.6130048632621765, Val Accuracy - 0.812
Epoch 120, Train Loss - 0.011819654144346714, Val Loss - 0.6150628924369812, Val Accuracy - 0.809
Epoch 140, Train Loss - 0.0118984030559659, Val Loss - 0.6284157037734985, Val Accuracy - 0.806
Epoch 160, Train Loss - 0.010907652787864208, Val Loss - 0.624652087688446, Val Accuracy - 0.804
Epoch 180, Train Loss - 0.009696302004158497, Val Loss - 0.6392356157302856, Val Accuracy - 0.804
Epoch 200, Train Loss - 0.00854632

Peturbing graph...:   0%|          | 0/1055 [00:00<?, ?it/s]

In [9]:
amts

defaultdict(int,
            {(49, 2603): 10,
             (115, 2644): 10,
             (86, 2602): 10,
             (131, 2643): 9,
             (58, 832): 10,
             (76, 167): 9,
             (76, 2438): 10,
             (50, 2482): 9,
             (131, 358): 1,
             (58, 654): 1,
             (58, 2378): 1,
             (58, 1643): 1,
             (58, 1390): 1,
             (58, 2675): 1,
             (58, 1454): 1,
             (68, 2672): 1,
             (58, 198): 1,
             (58, 837): 1,
             (82, 2603): 9,
             (131, 1018): 1,
             (58, 1559): 1,
             (58, 1418): 1,
             (58, 1322): 1,
             (58, 2549): 1,
             (58, 585): 9,
             (72, 378): 1,
             (34, 2682): 1,
             (101, 2513): 1,
             (131, 762): 9,
             (66, 1454): 1,
             (58, 2259): 1,
             (58, 884): 1,
             (58, 247): 1,
             (57, 1437): 9,
             (58, 956): 1,
    

In [10]:
print(len(amts))
sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=True)
sorted_keys = [key for key, value in sorted_list]

2107


In [11]:
edges_to_add = sorted_keys

In [12]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [13]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.20
budget = int(ptb_rate * initial_edge_count)

In [14]:
print(len(edges_to_add))
print(budget)

2107
527


In [15]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [16]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [17]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.1000
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Final Accuracy: 0.8050
Accuracy Drop: -0.0020
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0500
Ran out of candidate edges in Phase 1. Moving to Phase 2.
Phase 1: Added 0 edges out of requested 263.


Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Final Accuracy: 0.8090
Accuracy Drop: -0.0060


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
